In [2]:
# import independencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [84]:
#Reading sample file into a dataframe
file_path = Path('Resources/sample_covid_dataset.csv')
cases_df = pd.read_csv(file_path)
cases_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,_id,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,...,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,0,0,9,9,353345,353346,353346,2020-12-14T00:00:00,2020-12-20T00:00:00,2020-12-20T00:00:00,...,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,1,1,15,15,35074,35075,35075,2020-11-08T00:00:00,2020-11-09T00:00:00,2020-11-09T00:00:00,...,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
2,2,2,16,16,131940,131941,131941,2021-02-03T00:00:00,2021-02-05T00:00:00,2021-02-05T00:00:00,...,Resolved,Yes,2246,Niagara Region Public Health Department,1815 Sir Isaac Brock Way,Thorold,L2V 4T7,www.niagararegion.ca/health,43.116537,-79.241220
3,3,3,24,24,205768,205769,205769,2020-12-24T00:00:00,2020-12-29T00:00:00,2020-12-29T00:00:00,...,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
4,4,4,26,26,387017,387018,387018,2021-01-21T00:00:00,2021-01-23T00:00:00,2021-01-23T00:00:00,...,Resolved,Yes,2246,Niagara Region Public Health Department,1815 Sir Isaac Brock Way,Thorold,L2V 4T7,www.niagararegion.ca/health,43.116537,-79.241220


In [85]:
# Check the columns

cases_df.columns


Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', '_id', 'Row_ID', 'Accurate_Episode_Date',
       'Case_Reported_Date', 'Test_Reported_Date', 'Specimen_Date',
       'Age_Group', 'Client_Gender', 'Case_AcquisitionInfo', 'Outcome1',
       'Outbreak_Related', 'Reporting_PHU_ID', 'Reporting_PHU',
       'Reporting_PHU_Address', 'Reporting_PHU_City',
       'Reporting_PHU_Postal_Code', 'Reporting_PHU_Website',
       'Reporting_PHU_Latitude', 'Reporting_PHU_Longitude'],
      dtype='object')

In [86]:
# Drop unrelated columns from dataframe

coumns_to_drop=['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1','Unnamed: 0.1.1.1','Unnamed: 0.1.1.1.1', '_id', 'Row_ID', 
                'Case_Reported_Date','Test_Reported_Date','Specimen_Date', 'Case_AcquisitionInfo','Reporting_PHU','Reporting_PHU_Address', 
                'Reporting_PHU_City' , 'Reporting_PHU_Postal_Code',  'Reporting_PHU_Website',
                'Reporting_PHU_Latitude', 'Reporting_PHU_Longitude'
               ]
cleaned_cases_df = cases_df.drop(columns= coumns_to_drop)


In [87]:
cleaned_cases_df.columns

Index(['Accurate_Episode_Date', 'Age_Group', 'Client_Gender', 'Outcome1',
       'Outbreak_Related', 'Reporting_PHU_ID'],
      dtype='object')

In [88]:
# Rename columns to short version

cleaned_cases_df = cleaned_cases_df.rename(columns={"Accurate_Episode_Date": "Date",'Client_Gender':"Gender",
                                                  'Outcome1':'Outcome','Outbreak_Related': 'Outbreak',
                                                   'Reporting_PHU_ID':'PHU_id'})
cleaned_cases_df.columns

Index(['Date', 'Age_Group', 'Gender', 'Outcome', 'Outbreak', 'PHU_id'], dtype='object')

In [89]:
cleaned_cases_df

,Date,Age_Group,Gender,Outcome,Outbreak,PHU_id
0,2020-12-14T00:00:00,70s,FEMALE,Resolved,Yes,3895
1,2020-11-08T00:00:00,<20,FEMALE,Resolved,Yes,3895
2,2021-02-03T00:00:00,20s,MALE,Resolved,Yes,2246
3,2020-12-24T00:00:00,20s,MALE,Resolved,Yes,3895
4,2021-01-21T00:00:00,90+,FEMALE,Resolved,Yes,2246
...,...,...,...,...,...,...
13519,2021-03-15T00:00:00,20s,FEMALE,Resolved,Yes,2253
13520,2021-01-06T00:00:00,80s,FEMALE,Resolved,Yes,2261
13521,2021-02-07T00:00:00,80s,FEMALE,Resolved,Yes,3895
13522,2021-01-17T00:00:00,40s,FEMALE,Resolved,Yes,2253


In [90]:
#Converting date column

cleaned_cases_df['Date'] = pd.to_datetime(cleaned_cases_df['Date'], errors='coerce')

In [91]:
#Breaking Date column  to week, month and year 

cleaned_cases_df['Week'] = cleaned_cases_df['Date'].dt.week
cleaned_cases_df['Month'] = cleaned_cases_df['Date'].dt.month
cleaned_cases_df['Year'] = cleaned_cases_df['Date'].dt.year

C:\Users\16472\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [92]:
cleaned_cases_df

,Date,Age_Group,Gender,Outcome,Outbreak,PHU_id,Week,Month,Year
0,2020-12-14,70s,FEMALE,Resolved,Yes,3895,51,12,2020
1,2020-11-08,<20,FEMALE,Resolved,Yes,3895,45,11,2020
2,2021-02-03,20s,MALE,Resolved,Yes,2246,5,2,2021
3,2020-12-24,20s,MALE,Resolved,Yes,3895,52,12,2020
4,2021-01-21,90+,FEMALE,Resolved,Yes,2246,3,1,2021
...,...,...,...,...,...,...,...,...,...
13519,2021-03-15,20s,FEMALE,Resolved,Yes,2253,11,3,2021
13520,2021-01-06,80s,FEMALE,Resolved,Yes,2261,1,1,2021
13521,2021-02-07,80s,FEMALE,Resolved,Yes,3895,5,2,2021
13522,2021-01-17,40s,FEMALE,Resolved,Yes,2253,2,1,2021


In [93]:
# Drop Date column

cleaned_cases_df = cleaned_cases_df.drop(columns=['Date'])

In [94]:
cleaned_cases_df 

,Age_Group,Gender,Outcome,Outbreak,PHU_id,Week,Month,Year
0,70s,FEMALE,Resolved,Yes,3895,51,12,2020
1,<20,FEMALE,Resolved,Yes,3895,45,11,2020
2,20s,MALE,Resolved,Yes,2246,5,2,2021
3,20s,MALE,Resolved,Yes,3895,52,12,2020
4,90+,FEMALE,Resolved,Yes,2246,3,1,2021
...,...,...,...,...,...,...,...,...
13519,20s,FEMALE,Resolved,Yes,2253,11,3,2021
13520,80s,FEMALE,Resolved,Yes,2261,1,1,2021
13521,80s,FEMALE,Resolved,Yes,3895,5,2,2021
13522,40s,FEMALE,Resolved,Yes,2253,2,1,2021


In [106]:
# check the count of each PHU

PHU_count =cleaned_cases_df.PHU_id.value_counts()
PHU_count

3895    3581
2253    2077
2270     943
2251     808
2246     726
2268     700
2237     630
2230     609
2265     516
2236     472
2244     389
2260     383
2266     273
2262     193
4913     155
2258     118
2234     111
2242      96
2243      95
2261      87
2227      71
5183      70
2235      67
2249      52
2240      49
2241      47
2238      37
2255      36
2256      32
2233      31
2257      28
2247      20
2226      12
2263      10
Name: PHU_id, dtype: int64

In [98]:
# Check the count of cases based on Gender

Gender_count =cleaned_cases_df.Gender.value_counts()
Gender_count

FEMALE            7321
MALE              6093
UNSPECIFIED        108
GENDER DIVERSE       2
Name: Gender, dtype: int64

In [99]:
# Check the count of cases based on Age group

age_count =cleaned_cases_df.Age_Group.value_counts()
age_count

20s    1983
50s    1981
40s    1943
30s    1846
<20    1458
80s    1321
60s    1154
90+    1038
70s     800
Name: Age_Group, dtype: int64

In [104]:
# Grouped cases by PHU, Week, month and year

cases_per_PHU_df = cleaned_cases_df.groupby(['PHU_id','Week', 'Month','Year']).count()
cases_per_PHU_df

Age_Group  Gender  Outcome  Outbreak
PHU_id Week Month Year                                      
2226   1    1     2021          1       1        1         1
       2    1     2021          1       1        1         1
       4    1     2021          2       2        2         2
       12   3     2021          1       1        1         1
       13   3     2020          1       1        1         1
...                           ...     ...      ...       ...
5183   50   12    2020          2       2        2         2
       51   12    2020          3       3        3         3
       52   12    2020          2       2        2         2
       53   1     2021          6       6        6         6
            12    2020          3       3        3         3

[1239 rows x 4 columns]

In [105]:
# count cases by PHU

count_by_PHU = weekly_cases_per_unit_df.groupby(['PHU_id']).sum()
count_by_PHU

,Age_Group,Gender,Outcome,Outbreak
PHU_id,,,,
2226,12,12,12,12
2227,71,71,71,71
2230,609,609,609,609
2233,31,31,31,31
2234,111,111,111,111
2235,67,67,67,67
2236,472,472,472,472
2237,630,630,630,630
2238,37,37,37,37
